In [3]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install bs4

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Building dependency tree       
Reading state infor

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

In [5]:
import json
import numpy as np
import pandas as pd         
import ctypes

In [6]:
hospitals = pd.read_csv('https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/GA_Hospitals.csv')
hospitals = hospitals.fillna('')
hospitals['Med_center'] = hospitals['Med_center'] + " " + hospitals['Unnamed: 2']
hospitals = hospitals.drop('Unnamed: 2', axis=1)

baseurl = 'https://freida.ama-assn.org/institution/'
URLS = []
for index, i in hospitals.iterrows():
  URLS.append(str(baseurl + str(i['H.No.'])))
hospitals['URLS'] = URLS

hospitals.head()

,H.No.,Med_center,URLS
0,120127,Abraham Infectious Disease Associates,https://freida.ama-assn.org/institution/120127
1,128070,AG Rhodes Homes,https://freida.ama-assn.org/institution/128070
2,129528,Air Methods Air Life Georgia,https://freida.ama-assn.org/institution/129528
3,129505,Albany Arthritis and Orthopedic Center,https://freida.ama-assn.org/institution/129505
4,120063,Alfred E Beach High School (APEX Program by G...,https://freida.ama-assn.org/institution/120063


In [7]:
def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument("enable-automation")
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-browser-side-navigation")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    driver = webdriver.Chrome(options=options)

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

    return driver

def pagesource(url, driver):
    driver = driver
    driver.get(url)
    time.sleep(10)
    soup = BeautifulSoup(driver.page_source)
    #driver.close()
    return soup



In [8]:
d = driversetup()
s = pagesource('https://freida.ama-assn.org/institution/120139', d)
a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
for i in range(len(a)):
  name = a[i].findAll('small', class_='bold')[0].text
  relationship = a[i].findAll('div', class_='')[0].text
  if 'Major' in relationship:
    print(name, ': Major')
  if 'Limited' in relationship:
    print(name, ': Limited')
#mcl.append(name)

Emory Univ Sch of Med : Major
Morehouse Sch of Med : Limited
West Virginia Sch of Osteopathic Med : Limited


In [ ]:
# d = driversetup()
# relationship = []
# for index, i in hospitals.iterrows():
#   url = i['URLS']
#   s = pagesource(url, d)
#   med_school = list(s.findAll('small', class_="bold"))
#   print(i.Med_center, index, relationship, len(med_school))
#   for i in range(len(med_school)):
#     a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
    
#     for i in range(len(a)):
#       rel = a[i].findAll('div', class_='')[0].text
#       if rel not in relationship:
#         print(med_school[i].text, rel)
#         relationship.append(rel)
# relationship

In [27]:
mc = []
grad = []
mcl = []
add = []
d = driversetup()
for index, i in hospitals.iterrows():
  url = i['URLS']
  #url = 'https://freida.ama-assn.org/institution/120119'
  s = pagesource(url, d)
  med_school = list(s.findAll('small', class_="bold"))
  address = list(s.findAll('div', class_="details__address ng-star-inserted")) 
  if len(med_school) == 0:
    mc.append("")
    mcl.append("")
    grad.append("")
  elif len(med_school) == 1:
    a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
    name = a[0].findAll('small', class_='bold')[0].text
    relationship = a[0].findAll('div', class_='')[0].text
    print(name, relationship)
    if 'Limited' in relationship:
      mcl.append(name)
      mc.append("")
      grad.append("")
    elif 'Major' in relationship:
      mc.append(name)
      mcl.append("")
      grad.append("")
    elif 'Graduate' in relationship:
      mc.append("")
      mcl.append("")
      grad.append(name)
    else:
      print("NEW RELATIONSHIP FOUND : ", name)
      break
  elif len(med_school) > 1:   
      major = []
      limited = []
      graduate = []
      a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
      for i in range(len(a)):
        name = a[i].findAll('small', class_='bold')[0].text
        relationship = a[i].findAll('div', class_='')[0].text
        print(i, name, relationship)
        if 'Limited' in relationship:
          limited.append(name)
        elif 'Graduate' in relationship:
          graduate.append(name)
        elif 'Major' in relationship:
          major.append(name)
        else:
          print("New relationship found")
          break 
      if major == []:
        major = ''
      if limited == []:
        limited = ''
      if grad == []:
        grad = ''
      mc.append(major)
      mcl.append(limited)
      grad.append(graduate)
  if len(address) != 0:
    address[0].br.string = ', '
    add.append(address[0].text)
  else:
    add.append("")
  print(index, 'mc', len(mc), mc[index], 'mcl', len(mcl), mcl[index], 'grad', len(grad), grad[index], 'add', len(add), add[index])
  if len(mc) > (index+1):
    break



0 mc 1  mcl 1  grad 1  add 1 2024 E Pinetree Blvd, Thomasville, GA 31792
1 mc 2  mcl 2  grad 2  add 2 3715 Northside Pkwy, Atlanta, GA 30327
2 mc 3  mcl 3  grad 3  add 3 635 Regional Airport Rd, Carrollton, GA 30117
3 mc 4  mcl 4  grad 4  add 4 2100 Palmyra Rd, Albany, GA 31701
4 mc 5  mcl 5  grad 5  add 5 3001 Hopkins St, Savannah, GA 31405
5 mc 6  mcl 6  grad 6  add 6 1120 15th St, Augusta, GA 30912
Morehouse Sch of Med Relationship: Limited
6 mc 7  mcl 7 Morehouse Sch of Med grad 7  add 7 250 Williams St NW, Atlanta, GA 30303
7 mc 8  mcl 8  grad 8  add 8 112 Mimosa Dr, Thomasville, GA 31792
8 mc 9  mcl 9  grad 9  add 9 915 Gordon Ave, Thomasville, GA 31799
9 mc 10  mcl 10  grad 10  add 10 2705 E Pinetree Blvd, Thomasville, GA 31792
10 mc 11  mcl 11  grad 11  add 11 961 Smoky Mountain Springs, Gainesville, GA 30501
11 mc 12  mcl 12  grad 12  add 12 895 Canton Rd, Marietta, GA 30060
12 mc 13  mcl 13  grad 13  add 13 3193 Howell Mill Rd, Atlanta, GA 30327
13 mc 14  mcl 14  grad 14  add

In [30]:
# print(len(mc),len(mcl),len(grad),len(add))
# hospitals['Affiliation_Major'] = mc
# hospitals['Affiliation_Limited'] = mcl
# hospitals['Affiliation_Graduate'] = grad
# hospitals['Medical_Center_Address'] = add

# hospitals.iloc[[40]]

hospitals.to_csv('GA_crosswalk.csv')